# Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()

In [5]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers

In [7]:
tf.__version__

'1.10.1'

In [23]:
BATCH_SIZE = 256
EPOCHS = 3
LEARNING_RATE = 0.001

In [10]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/TF/'

In [11]:
%%time
train_clean = pd.read_feather('tmp/taxi-train-clean')
test_clean = pd.read_feather('tmp/taxi-test-clean')
test_key = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

/home/szelee/anaconda3/envs/tf/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


CPU times: user 11.4 s, sys: 9.57 s, total: 21 s
Wall time: 21 s


In [13]:
train_clean.shape, test_clean.shape

((54062903, 46), (9914, 45))

In [14]:
%%time
train_df, validation_df = train_test_split(train_clean, test_size=0.10, random_state=1)

train_labels = train_df['fare_amount'].values
train_df = train_df.drop(['fare_amount'], axis=1)
validation_labels = validation_df['fare_amount'].values
validation_df = validation_df.drop(['fare_amount'], axis=1)

CPU times: user 1min 14s, sys: 21.3 s, total: 1min 35s
Wall time: 1min 35s


In [15]:
train_df.shape, train_labels.shape, validation_df.shape, validation_labels.shape

((48656612, 45), (48656612,), (5406291, 45), (5406291,))

In [16]:
%%time
scaler = MinMaxScaler()

train_df_scaled = scaler.fit_transform(train_df)
validation_df_scaled = scaler.transform(validation_df)
test_scaled = scaler.transform(test_clean)

In [24]:
model = Sequential()

model.add(Dense(256, activation='relu', input_dim=train_df_scaled.shape[1], activity_regularizer=regularizers.l1(0.01)))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(8, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1))

adam = optimizers.Adam(lr=LEARNING_RATE)
model.compile(loss='mse', optimizer=adam, metrics=['mse'])

In [25]:
%%time
history = model.fit(x=train_df_scaled, y=train_labels, batch_size=BATCH_SIZE, epochs=EPOCHS, 
                    verbose=1, validation_data=(validation_df_scaled, validation_labels), 
                    shuffle=True)

Train on 48656612 samples, validate on 5406291 samples
Epoch 1/3
48656612/48656612 [==============================] - 2967s 61us/step - loss: 18.0792 - mean_squared_error: 17.6985 - val_loss: 20.1034 - val_mean_squared_error: 19.6909
Epoch 2/3
48656612/48656612 [==============================] - 2960s 61us/step - loss: 17.4469 - mean_squared_error: 16.9970 - val_loss: 35.0345 - val_mean_squared_error: 34.5715
Epoch 3/3
48656612/48656612 [==============================] - 2940s 60us/step - loss: 17.3539 - mean_squared_error: 16.8171 - val_loss: 23.9528 - val_mean_squared_error: 23.3449
CPU times: user 8h 8min 35s, sys: 2h 26min 17s, total: 10h 34min 53s
Wall time: 2h 27min 48s


In [26]:
prediction1 = model.predict(test_scaled, batch_size=128, verbose=1)

9914/9914 [==============================] - 0s 42us/step


In [ ]:
submission = pd.DataFrame(
    {'key': test_key.key, 'fare_amount': prediction},
    columns = ['key', 'fare_amount'])

submission.to_csv(f'{CSV_PATH}tf1.csv', index = False)